<a href="https://colab.research.google.com/github/yonghoonlee-1/Cyberbullying-Detection/blob/main/210410_210213_kfold(5)_shuffle_cybully_formspr_CNN%2Bbilstm_with_Glove_rev2%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
!rm -rf /content/gdrive
!ln -s "/gdrive/My Drive" /content/gdrive
%cd /content/gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [2]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [3]:
formspring=pd.read_csv("/content/gdrive/formspring_data.csv", encoding='latin1')

In [4]:
formspring.head(10)

,sentence,sentiment
0,Q: what&#039;s your favorite song? :D<br>A: I ...,non-bully
1,Q: <3<br>A: </3 ? haha jk! <33,non-bully
2,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,non-bully
3,Q: (:<br>A: ;(,non-bully
4,Q: ******************MEOWWW*******************...,non-bully
5,Q: any makeup tips? i suck at doing my makeup ...,non-bully
6,Q: Apriiiiiiiiiiiill!!! I miss uuuu! It&#039;s...,non-bully
7,Q: Are you a morning or night person?<br>A: Ni...,non-bully
8,Q: are you a trusting person?<br>A: alreadi an...,non-bully
9,Q: are you a trusting person?<br>A: Yes veryy ...,non-bully


In [5]:
data = formspring[['sentence','sentiment']]

In [6]:
import string
data['sentence'] = data['sentence'].apply(lambda x: x.replace('Q:',' ').replace('A:',' ').replace('<br>',' '))
data['sentence'] = data['sentence'].apply(lambda x: x.lower())
data['sentence'] = data['sentence'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))
print(data[ data['sentiment'] == 'non-bully'].size)
print(data[ data['sentiment'] == 'bully'].size)

23994
1552


In [7]:
def text_to_wordlist(text, remove_stopwords=True):    

    text = text.lower().split()
    
    if remove_stopwords:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)


    return(text)

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
data['sentence'] = data.apply(lambda row: text_to_wordlist(row.sentence).split(), axis=1)

In [10]:
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(data['sentence'].values)
X = tokenizer.texts_to_sequences(data['sentence'].values)
X = pad_sequences(X, maxlen=62)

In [11]:
Y = pd.get_dummies(data['sentiment']).values

In [12]:
print(X.shape)

(12773, 62)


In [13]:
from sklearn.model_selection import KFold

skf = KFold(n_splits=5, shuffle=True)
accuracy = []
precision = []
recall = []
f1_score = []

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/glove.twitter.27B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [15]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [16]:
from keras.layers import Bidirectional, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D 
from sklearn import metrics
from sklearn.utils import shuffle

X, Y = shuffle(X, Y, random_state=42)


embed_dim = 50
lstm_out = 50
for train, validation in skf.split(X, Y):
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, embed_dim, weights=[embedding_matrix], input_length = X.shape[1]))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(lstm_out, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.fit(X[train], Y[train], epochs=10, batch_size=128)
    k_accuracy = '%.4f' % (model.evaluate(X[validation], Y[validation])[1])
    predict_test = (model.predict(X[validation]) >= 0.5) 
    k_precision = metrics.precision_score(Y[validation], predict_test, average=None)
    k_recall = metrics.recall_score(Y[validation], predict_test, average=None)
    k_f1_score = metrics.f1_score(Y[validation], predict_test, average=None)
    accuracy.append(k_accuracy)
    precision.append(k_precision)
    recall.append(k_recall)
    f1_score.append(k_f1_score)

Epoch 1/10
80/80 [==============================] - 7s 20ms/step - loss: 0.2965 - accuracy: 0.9124
Epoch 2/10
80/80 [==============================] - 1s 17ms/step - loss: 0.1794 - accuracy: 0.9441
Epoch 3/10
80/80 [==============================] - 1s 17ms/step - loss: 0.1478 - accuracy: 0.9492
Epoch 4/10
80/80 [==============================] - 1s 18ms/step - loss: 0.1265 - accuracy: 0.9559
Epoch 5/10
80/80 [==============================] - 1s 18ms/step - loss: 0.1101 - accuracy: 0.9583
Epoch 6/10
80/80 [==============================] - 1s 18ms/step - loss: 0.1012 - accuracy: 0.9620
Epoch 7/10
80/80 [==============================] - 1s 19ms/step - loss: 0.0909 - accuracy: 0.9658
Epoch 8/10
80/80 [==============================] - 2s 19ms/step - loss: 0.0655 - accuracy: 0.9759
Epoch 9/10
80/80 [==============================] - 2s 19ms/step - loss: 0.0688 - accuracy: 0.9737
Epoch 10/10
80/80 [==============================] - 1s 5ms/step - loss: 0.1587 - accuracy: 0.9534
Epoch 1/10

In [17]:
def average(list):
    return (sum(list) / len(list))

In [18]:
accuracy = list(map(float, accuracy))

In [19]:
print(average(accuracy))

0.94848


In [20]:
print(average(precision))

[0.60075219 0.96753927]


In [21]:
print(average(recall))

[0.4904009 0.9779823]


In [22]:
print(average(f1_score))

[0.53529034 0.97270389]
